In [ ]:
# mounting drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# change to current directory

%cd /content/drive/Shareddrives/DLNN_ProjF

/content/drive/Shareddrives/DLNN_ProjF


In [ ]:
# Bounding box buffer

BUFFER = 10

In [ ]:
# importing libraries

import pandas as pd
import numpy as np
import os, sys
import cv2
!pip install tensorlayer
import tensorlayer as tl

     |████████████████████████████████| 368kB 7.7MB/s 
     |████████████████████████████████| 3.3MB 27.7MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1
  Found existing installation: progressbar2 3.38.0
    Uninstalling progressbar2-3.38.0:
      Successfully uninstalled progressbar2-3.38.0


In [ ]:
# Loading the images and annotations from mpii dataset for processing
img_train_list, ann_train_list, img_test_list, ann_test_list = tl.files.load_mpii_pose_dataset('data/', True)

[TL] Load or Download MPII Human Pose > data/mpii_human_pose
[TL] [MPII] (annotation) mpii_human_pose_v1_u12_2 is nonexistent in data/mpii_human_pose
[TL] Downloading mpii_human_pose_v1_u12_2.zip...



100% (1507 of 1507) |####################| Elapsed Time: 0:00:05 ETA:  00:00:00

[TL] Succesfully downloaded mpii_human_pose_v1_u12_2.zip 12340483 bytes.
[TL] Trying to extract zip file
[TL] ... Success!
[TL] [MPII] (images) mpii_human_pose_v1_u12_2 is nonexistent in data/mpii_human_pose
[TL] Downloading mpii_human_pose_v1.tar.gz...



100% (1475702 of 1475702) |##############| Elapsed Time: 1:10:19 ETA:  00:00:00

[TL] Succesfully downloaded mpii_human_pose_v1.tar.gz 12088943206 bytes.
[TL] Trying to extract tar file
[TL] ... Success!
[TL] reading annotations from mat file ...
[TL] reading images list ...
missing training image 040348287.jpg in data/mpii_human_pose/images (remove from img(ann)_train_list)
missing training image 002878268.jpg in data/mpii_human_pose/images (remove from img(ann)_train_list)
[TL] n_images: 24985 n_train_images: 18077 n_test_images: 6908
[TL] n_ann: 24985 n_train_ann: 18077 n_test_ann: 6908
[TL] n_people: 19919 n_train_people: 19919 n_test_people: 0


In [ ]:
# checking the annotations
ann_train_list[:10]

[[{'filename': '015601864.jpg',
   'head_rect': [627.0, 100.0, 706.0, 198.0],
   'is_visible': {'0': 1,
    '1': 1,
    '10': 1,
    '11': 1,
    '12': 1,
    '13': 1,
    '14': 1,
    '15': 1,
    '2': 1,
    '3': 0,
    '4': 1,
    '5': 1,
    '6': 0,
    '7': 1,
    '8': 0,
    '9': 0},
   'joint_pos': {0: [620.0, 394.0],
    1: [616.0, 269.0],
    2: [573.0, 185.0],
    3: [647.0, 188.0],
    4: [661.0, 221.0],
    5: [656.0, 231.0],
    6: [610.0, 187.0],
    7: [647.0, 176.0],
    8: [637.0201, 189.8183],
    9: [695.9799, 108.1817],
    10: [606.0, 217.0],
    11: [553.0, 161.0],
    12: [601.0, 167.0],
    13: [692.0, 185.0],
    14: [693.0, 240.0],
    15: [688.0, 313.0]},
   'train': 1},
  {'filename': '015601864.jpg',
   'head_rect': [841.0, 145.0, 902.0, 228.0],
   'is_visible': {'0': 1,
    '1': 1,
    '10': 1,
    '11': 1,
    '12': 0,
    '13': 1,
    '14': 1,
    '15': 1,
    '2': 0,
    '3': 1,
    '4': 1,
    '5': 1,
    '6': 0,
    '7': 0,
    '8': 0,
    '9': 0},
  

In [ ]:
# load annotation list into a dataframe
df = pd.DataFrame(ann_train_list).stack().apply(pd.Series)
df = df.reset_index()

# rename joint presence column from their indices to their respective names
df = pd.concat((df, pd.DataFrame(df.is_visible.values.tolist())),axis=1)
df.rename(columns={'6': 'pelvis', '7': 'thorax', '8': 'upper_neck', '9': 'head_top', '0': 'r_ankle', '1': 'r_knee', '2': 'r_hip', '3': 'l_hip', '4': 'l_knee', '5': 'l_ankle', '10': 'r_wrist', '11': 'r_elbow', '12': 'r_shoulder', '13': 'l_shoulder', '14': 'l_elbow', '15': 'l_wrist' }, inplace=True)
df

,level_0,level_1,filename,train,head_rect,is_visible,joint_pos,pelvis,thorax,upper_neck,head_top,r_ankle,r_knee,r_hip,l_hip,l_knee,l_ankle,r_wrist,r_elbow,r_shoulder,l_shoulder,l_elbow,l_wrist
0,0,0,015601864.jpg,1,"[627.0, 100.0, 706.0, 198.0]","{'6': 0, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [610.0, 187.0], 7: [647.0, 176.0], 8: [637...",0,1,0,0,1,1,1,0,1,1,1,1,1,1,1,1
1,0,1,015601864.jpg,1,"[841.0, 145.0, 902.0, 228.0]","{'6': 0, '7': 0, '8': 0, '9': 0, '0': 1, '1': ...","{6: [979.0, 221.0], 7: [906.0, 190.0], 8: [912...",0,0,0,0,1,1,0,1,1,1,1,1,0,1,1,1
2,2,0,005808361.jpg,1,"[914.0, 79.0, 1031.0, 237.0]","{'6': 1, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [974.0, 446.0], 7: [985.0, 253.0], 8: [982...",1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1
3,2,1,005808361.jpg,1,"[427.0, 66.0, 549.0, 221.0]","{'6': 1, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [471.0, 512.0], 7: [463.0, 268.0], 8: [472...",1,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1
4,3,0,086617615.jpg,1,"[252.0, 187.0, 331.0, 266.0]","{'6': 0, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [248.0, 341.0], 7: [279.0, 263.0], 8: [277...",0,1,0,0,1,1,1,0,0,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19914,18074,0,072772110.jpg,1,"[506.0, 305.0, 548.0, 351.0]","{'6': 1, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [532.0, 455.0], 7: [531.0, 378.0], 8: [528...",1,1,0,0,1,1,1,1,1,1,0,0,1,1,1,1
19915,18075,0,039361034.jpg,1,"[406.0, 225.0, 449.0, 268.0]","{'6': 0, '7': 0, '8': 0, '9': 0, '0': 1, '1': ...","{6: [525.0, 279.0], 7: [454.0, 252.0], 8: [451...",0,0,0,0,1,1,0,1,1,1,0,1,0,1,1,1
19916,18076,0,084761779.jpg,1,"[287.0, 130.0, 324.0, 163.0]","{'6': 1, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [334.0, 225.0], 7: [312.0, 173.0], 8: [310...",1,1,0,0,1,1,1,1,0,1,0,0,1,1,0,0
19917,18076,1,084761779.jpg,1,"[333.0, 137.0, 361.0, 162.0]","{'6': 1, '7': 0, '8': 0, '9': 0, '0': 1, '1': ...","{6: [391.0, 221.0], 7: [365.0, 172.0], 8: [356...",1,0,0,0,1,1,1,1,0,1,0,0,0,1,0,0


In [ ]:
# deserializing joint coordinates to their respective columns

# cols = ['joint_pos_pelvis', 'joint_pos_thorax', 'joint_pos_upper_neck', 'joint_pos_head_top', 'joint_pos_r_ankle', 'joint_pos_r_knee', 'joint_pos_r_hip', 'joint_pos_l_hip', 'joint_pos_l_knee', 'joint_pos_l_ankle', 'joint_pos_r_wrist', 'joint_pos_r_elbow', 'joint_pos_r_shoulder', 'joint_pos_l_shoulder', 'joint_pos_l_elbow', 'joint_pos_l_wrist']
df = pd.concat((df, pd.DataFrame(df.joint_pos.values.tolist())),axis=1)
df.rename(columns={6: 'joint_pos_pelvis', 7: 'joint_pos_thorax', 8: 'joint_pos_upper_neck', 9: 'joint_pos_head_top', 0: 'joint_pos_r_ankle', 1: 'joint_pos_r_knee', 2: 'joint_pos_r_hip', 3: 'joint_pos_l_hip', 4: 'joint_pos_l_knee', 5: 'joint_pos_l_ankle', 10: 'joint_pos_r_wrist', 11: 'joint_pos_r_elbow', 12: 'joint_pos_r_shoulder', 13: 'joint_pos_l_shoulder', 14: 'joint_pos_l_elbow', 15: 'joint_pos_l_wrist' }, inplace=True)
df


,level_0,level_1,filename,train,head_rect,is_visible,joint_pos,pelvis,thorax,upper_neck,head_top,r_ankle,r_knee,r_hip,l_hip,l_knee,l_ankle,r_wrist,r_elbow,r_shoulder,l_shoulder,l_elbow,l_wrist,joint_pos_pelvis,joint_pos_thorax,joint_pos_upper_neck,joint_pos_head_top,joint_pos_r_ankle,joint_pos_r_knee,joint_pos_r_hip,joint_pos_l_hip,joint_pos_l_knee,joint_pos_l_ankle,joint_pos_r_wrist,joint_pos_r_elbow,joint_pos_r_shoulder,joint_pos_l_shoulder,joint_pos_l_elbow,joint_pos_l_wrist
0,0,0,015601864.jpg,1,"[627.0, 100.0, 706.0, 198.0]","{'6': 0, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [610.0, 187.0], 7: [647.0, 176.0], 8: [637...",0,1,0,0,1,1,1,0,1,1,1,1,1,1,1,1,"[610.0, 187.0]","[647.0, 176.0]","[637.0201, 189.8183]","[695.9799, 108.1817]","[620.0, 394.0]","[616.0, 269.0]","[573.0, 185.0]","[647.0, 188.0]","[661.0, 221.0]","[656.0, 231.0]","[606.0, 217.0]","[553.0, 161.0]","[601.0, 167.0]","[692.0, 185.0]","[693.0, 240.0]","[688.0, 313.0]"
1,0,1,015601864.jpg,1,"[841.0, 145.0, 902.0, 228.0]","{'6': 0, '7': 0, '8': 0, '9': 0, '0': 1, '1': ...","{6: [979.0, 221.0], 7: [906.0, 190.0], 8: [912...",0,0,0,0,1,1,0,1,1,1,1,1,0,1,1,1,"[979.0, 221.0]","[906.0, 190.0]","[912.4915, 190.6586]","[830.5085, 182.3414]","[895.0, 293.0]","[910.0, 279.0]","[945.0, 223.0]","[1012.0, 218.0]","[961.0, 315.0]","[960.0, 403.0]","[871.0, 304.0]","[883.0, 229.0]","[888.0, 174.0]","[924.0, 206.0]","[1013.0, 203.0]","[955.0, 263.0]"
2,2,0,005808361.jpg,1,"[914.0, 79.0, 1031.0, 237.0]","{'6': 1, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [974.0, 446.0], 7: [985.0, 253.0], 8: [982...",1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,"[974.0, 446.0]","[985.0, 253.0]","[982.7591, 235.9694]","[962.2409, 80.0306]","[804.0, 711.0]","[816.0, 510.0]","[908.0, 438.0]","[1040.0, 454.0]","[906.0, 528.0]","[883.0, 707.0]","[869.0, 214.0]","[798.0, 340.0]","[902.0, 253.0]","[1067.0, 253.0]","[1167.0, 353.0]","[1142.0, 478.0]"
3,2,1,005808361.jpg,1,"[427.0, 66.0, 549.0, 221.0]","{'6': 1, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [471.0, 512.0], 7: [463.0, 268.0], 8: [472...",1,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,"[471.0, 512.0]","[463.0, 268.0]","[472.4664, 220.8573]","[503.5336, 66.1427]","[667.0, 633.0]","[675.0, 462.0]","[567.0, 519.0]","[375.0, 504.0]","[543.0, 476.0]","[532.0, 651.0]","[702.0, 267.0]","[721.0, 386.0]","[584.0, 256.0]","[341.0, 280.0]","[310.0, 432.0]","[372.0, 496.0]"
4,3,0,086617615.jpg,1,"[252.0, 187.0, 331.0, 266.0]","{'6': 0, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [248.0, 341.0], 7: [279.0, 263.0], 8: [277...",0,1,0,0,1,1,1,0,0,1,1,1,1,1,1,0,"[248.0, 341.0]","[279.0, 263.0]","[277.021, 268.7786]","[305.979, 184.2214]","[301.0, 461.0]","[305.0, 375.0]","[201.0, 340.0]","[294.0, 342.0]","[335.0, 370.0]","[331.0, 455.0]","[328.0, 354.0]","[260.0, 335.0]","[244.0, 261.0]","[314.0, 264.0]","[327.0, 320.0]","[362.0, 346.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19914,18074,0,072772110.jpg,1,"[506.0, 305.0, 548.0, 351.0]","{'6': 1, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [532.0, 455.0], 7: [531.0, 378.0], 8: [528...",1,1,0,0,1,1,1,1,1,1,0,0,1,1,1,1,"[532.0, 455.0]","[531.0, 378.0]","[528.9869, 352.8365]","[525.0131, 303.1635]","[562.0, 617.0]","[558.0, 539.0]","[553.0, 452.0]","[511.0, 458.0]","[529.0, 536.0]","[541.0, 610.0]","[558.0, 411.0]","[565.0, 393.0]","[572.0, 377.0]","[490.0, 378.0]","[470.0, 411.0]","[461.0, 414.0]"
19915,18075,0,039361034.jpg,1,"[406.0, 225.0, 449.0, 268.0]","{'6': 0, '7': 0, '8': 0, '9': 0, '0': 1, '1': ...","{6: [525.0, 279.0], 7: [454.0, 252.0], 8: [451...",0,0,0,0,1,1,0,1,1,1,0,1,0,1,1,1,"[525.0, 279.0]","[454.0, 252.0]","[451.3169, 251.4431]","[403.6831, 241.5569]","[496.0, 377.0]","[466.0, 322.0]","[510.0, 278.0]","[540.0, 280.0]","[558.0, 331.0]","[596.0, 368.0]","[378.0, 311.0]","[404.0, 288.0]","[441.0, 253.0]","[466.0, 251.0]","[436.0, 293.0]","[410.0, 325.0]"
19916,18076,0,084761779.jpg,1

In [ ]:
#splitting joint positions into x,y coordinates

df = pd.concat((df, pd.DataFrame(df.head_rect.values.tolist())),axis=1)
df.rename(columns={0: 'head_rect_x1', 1: 'head_rect_y1', 2: 'head_rect_x2', 3: 'head_rect_y2'}, inplace=True)
df = pd.concat((df, pd.DataFrame(df.joint_pos_head_top.values.tolist())),axis=1)
df.rename(columns={0: 'joint_pos_head_top_x', 1: 'joint_pos_head_top_y'}, inplace=True)
df = pd.concat((df, pd.DataFrame(df.joint_pos_r_ankle.values.tolist())),axis=1)
df.rename(columns={0: 'joint_pos_r_ankle_x', 1: 'joint_pos_r_ankle_y'}, inplace=True)
df = pd.concat((df, pd.DataFrame(df.joint_pos_r_knee.values.tolist())),axis=1)
df.rename(columns={0: 'joint_pos_r_knee_x', 1: 'joint_pos_r_knee_y'}, inplace=True)
df = pd.concat((df, pd.DataFrame(df.joint_pos_r_hip.values.tolist())),axis=1)
df.rename(columns={0: 'joint_pos_r_hip_x', 1: 'joint_pos_r_hip_y'}, inplace=True)
df = pd.concat((df, pd.DataFrame(df.joint_pos_l_hip.values.tolist())),axis=1)
df.rename(columns={0: 'joint_pos_l_hip_x', 1: 'joint_pos_l_hip_y'}, inplace=True)
df = pd.concat((df, pd.DataFrame(df.joint_pos_l_knee.values.tolist())),axis=1)
df.rename(columns={0: 'joint_pos_l_knee_x', 1: 'joint_pos_l_knee_y'}, inplace=True)
df = pd.concat((df, pd.DataFrame(df.joint_pos_l_ankle.values.tolist())),axis=1)
df.rename(columns={0: 'joint_pos_l_ankle_x', 1: 'joint_pos_l_ankle_y'}, inplace=True)
df = pd.concat((df, pd.DataFrame(df.joint_pos_r_wrist.values.tolist())),axis=1)
df.rename(columns={0: 'joint_pos_r_wrist_x', 1: 'joint_pos_r_wrist_y'}, inplace=True)
df = pd.concat((df, pd.DataFrame(df.joint_pos_r_elbow.values.tolist())),axis=1)
df.rename(columns={0: 'joint_pos_r_elbow_x', 1: 'joint_pos_r_elbow_y'}, inplace=True)
df = pd.concat((df, pd.DataFrame(df.joint_pos_r_shoulder.values.tolist())),axis=1)
df.rename(columns={0: 'joint_pos_r_shoulder_x', 1: 'joint_pos_r_shoulder_y'}, inplace=True)
df = pd.concat((df, pd.DataFrame(df.joint_pos_l_shoulder.values.tolist())),axis=1)
df.rename(columns={0: 'joint_pos_l_shoulder_x', 1: 'joint_pos_l_shoulder_y'}, inplace=True)
df = pd.concat((df, pd.DataFrame(df.joint_pos_l_elbow.values.tolist())),axis=1)
df.rename(columns={0: 'joint_pos_l_elbow_x', 1: 'joint_pos_l_elbow_y'}, inplace=True)
df = pd.concat((df, pd.DataFrame(df.joint_pos_l_wrist.values.tolist())),axis=1)
df.rename(columns={0: 'joint_pos_l_wrist_x', 1: 'joint_pos_l_wrist_y'}, inplace=True)

df

,level_0,level_1,filename,train,head_rect,is_visible,joint_pos,pelvis,thorax,upper_neck,head_top,r_ankle,r_knee,r_hip,l_hip,l_knee,l_ankle,r_wrist,r_elbow,r_shoulder,l_shoulder,l_elbow,l_wrist,joint_pos_pelvis,joint_pos_thorax,joint_pos_upper_neck,joint_pos_head_top,joint_pos_r_ankle,joint_pos_r_knee,joint_pos_r_hip,joint_pos_l_hip,joint_pos_l_knee,joint_pos_l_ankle,joint_pos_r_wrist,joint_pos_r_elbow,joint_pos_r_shoulder,joint_pos_l_shoulder,joint_pos_l_elbow,joint_pos_l_wrist,head_rect_x1,head_rect_y1,head_rect_x2,head_rect_y2,joint_pos_head_top_x,joint_pos_head_top_y,joint_pos_r_ankle_x,joint_pos_r_ankle_y,joint_pos_r_knee_x,joint_pos_r_knee_y,joint_pos_r_hip_x,joint_pos_r_hip_y,joint_pos_l_hip_x,joint_pos_l_hip_y,joint_pos_l_knee_x,joint_pos_l_knee_y,joint_pos_l_ankle_x,joint_pos_l_ankle_y,joint_pos_r_wrist_x,joint_pos_r_wrist_y,joint_pos_r_elbow_x,joint_pos_r_elbow_y,joint_pos_r_shoulder_x,joint_pos_r_shoulder_y,joint_pos_l_shoulder_x,joint_pos_l_shoulder_y,joint_pos_l_elbow_x,joint_pos_l_elbow_y,joint_pos_l_wrist_x,joint_pos_l_wrist_y
0,0,0,015601864.jpg,1,"[627.0, 100.0, 706.0, 198.0]","{'6': 0, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [610.0, 187.0], 7: [647.0, 176.0], 8: [637...",0,1,0,0,1,1,1,0,1,1,1,1,1,1,1,1,"[610.0, 187.0]","[647.0, 176.0]","[637.0201, 189.8183]","[695.9799, 108.1817]","[620.0, 394.0]","[616.0, 269.0]","[573.0, 185.0]","[647.0, 188.0]","[661.0, 221.0]","[656.0, 231.0]","[606.0, 217.0]","[553.0, 161.0]","[601.0, 167.0]","[692.0, 185.0]","[693.0, 240.0]","[688.0, 313.0]",627.0,100.0,706.0,198.0,695.9799,108.1817,620.0,394.0,616.0,269.0,573.0,185.0,647.0,188.0,661.0,221.0,656.0,231.0,606.0,217.0,553.0,161.0,601.0,167.0,692.0,185.0,693.0,240.0,688.0,313.0
1,0,1,015601864.jpg,1,"[841.0, 145.0, 902.0, 228.0]","{'6': 0, '7': 0, '8': 0, '9': 0, '0': 1, '1': ...","{6: [979.0, 221.0], 7: [906.0, 190.0], 8: [912...",0,0,0,0,1,1,0,1,1,1,1,1,0,1,1,1,"[979.0, 221.0]","[906.0, 190.0]","[912.4915, 190.6586]","[830.5085, 182.3414]","[895.0, 293.0]","[910.0, 279.0]","[945.0, 223.0]","[1012.0, 218.0]","[961.0, 315.0]","[960.0, 403.0]","[871.0, 304.0]","[883.0, 229.0]","[888.0, 174.0]","[924.0, 206.0]","[1013.0, 203.0]","[955.0, 263.0]",841.0,145.0,902.0,228.0,830.5085,182.3414,895.0,293.0,910.0,279.0,945.0,223.0,1012.0,218.0,961.0,315.0,960.0,403.0,871.0,304.0,883.0,229.0,888.0,174.0,924.0,206.0,1013.0,203.0,955.0,263.0
2,2,0,005808361.jpg,1,"[914.0, 79.0, 1031.0, 237.0]","{'6': 1, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [974.0, 446.0], 7: [985.0, 253.0], 8: [982...",1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,"[974.0, 446.0]","[985.0, 253.0]","[982.7591, 235.9694]","[962.2409, 80.0306]","[804.0, 711.0]","[816.0, 510.0]","[908.0, 438.0]","[1040.0, 454.0]","[906.0, 528.0]","[883.0, 707.0]","[869.0, 214.0]","[798.0, 340.0]","[902.0, 253.0]","[1067.0, 253.0]","[1167.0, 353.0]","[1142.0, 478.0]",914.0,79.0,1031.0,237.0,962.2409,80.0306,804.0,711.0,816.0,510.0,908.0,438.0,1040.0,454.0,906.0,528.0,883.0,707.0,869.0,214.0,798.0,340.0,902.0,253.0,1067.0,253.0,1167.0,353.0,1142.0,478.0
3,2,1,005808361.jpg,1,"[427.0, 66.0, 549.0, 221.0]","{'6': 1, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [471.0, 512.0], 7: [463.0, 268.0], 8: [472...",1,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,"[471.0, 512.0]","[463.0, 268.0]","[472.4664, 220.8573]","[503.5336, 66.1427]","[667.0, 633.0]","[675.0, 462.0]","[567.0, 519.0]","[375.0, 504.0]","[543.0, 476.0]","[532.0, 651.0]","[702.0, 267.0]","[721.0, 386.0]","[584.0, 256.0]","[341.0, 280.0]","[310.0, 432.0]","[372.0, 496.0]",427.0,66.0,549.0,221.0,503.5336,66.1427,667.0,633.0,675.0,462.0,567.0,519.0,375.0,504.0,543.0,476.0,532.0,651.0,702.0,267.0,721.0,386.0,584.0,256.0,341.0,280.0,310.0,432.0,372.0,496.0
4,3,0,086617615.jpg,1,"[252.0, 187.0, 331.0, 266.0]","{'6': 0, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [248.0, 341.0], 7: [279.0, 263.0], 8: [277...",0,1,0,0,1,1,1,0,0,1,1,1,1,1,1,0,"[248.0, 341.0]","[279.0, 263.0]","[277.021, 268.7786]","[305.979, 184.2214]","[301.0, 461.0]","[305.0, 375.0]",

In [ ]:
# pruning list to contain data with at least 12 keypoints

col_list = ['head_top', 'r_ankle', 'r_knee', 'r_hip', 'l_hip', 'l_knee', 'l_ankle', 'r_wrist', 'r_elbow', 'r_shoulder', 'l_shoulder', 'l_elbow', 'l_wrist']
df['total'] = df[col_list].sum(axis=1)
df = df[df.total > 11]
df

,level_0,level_1,filename,train,head_rect,is_visible,joint_pos,pelvis,thorax,upper_neck,head_top,r_ankle,r_knee,r_hip,l_hip,l_knee,l_ankle,r_wrist,r_elbow,r_shoulder,l_shoulder,l_elbow,l_wrist,joint_pos_pelvis,joint_pos_thorax,joint_pos_upper_neck,joint_pos_head_top,joint_pos_r_ankle,joint_pos_r_knee,joint_pos_r_hip,joint_pos_l_hip,joint_pos_l_knee,joint_pos_l_ankle,joint_pos_r_wrist,joint_pos_r_elbow,joint_pos_r_shoulder,joint_pos_l_shoulder,joint_pos_l_elbow,joint_pos_l_wrist,head_rect_x1,head_rect_y1,head_rect_x2,head_rect_y2,joint_pos_head_top_x,joint_pos_head_top_y,joint_pos_r_ankle_x,joint_pos_r_ankle_y,joint_pos_r_knee_x,joint_pos_r_knee_y,joint_pos_r_hip_x,joint_pos_r_hip_y,joint_pos_l_hip_x,joint_pos_l_hip_y,joint_pos_l_knee_x,joint_pos_l_knee_y,joint_pos_l_ankle_x,joint_pos_l_ankle_y,joint_pos_r_wrist_x,joint_pos_r_wrist_y,joint_pos_r_elbow_x,joint_pos_r_elbow_y,joint_pos_r_shoulder_x,joint_pos_r_shoulder_y,joint_pos_l_shoulder_x,joint_pos_l_shoulder_y,joint_pos_l_elbow_x,joint_pos_l_elbow_y,joint_pos_l_wrist_x,joint_pos_l_wrist_y,total
2,2,0,005808361.jpg,1,"[914.0, 79.0, 1031.0, 237.0]","{'6': 1, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [974.0, 446.0], 7: [985.0, 253.0], 8: [982...",1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,"[974.0, 446.0]","[985.0, 253.0]","[982.7591, 235.9694]","[962.2409, 80.0306]","[804.0, 711.0]","[816.0, 510.0]","[908.0, 438.0]","[1040.0, 454.0]","[906.0, 528.0]","[883.0, 707.0]","[869.0, 214.0]","[798.0, 340.0]","[902.0, 253.0]","[1067.0, 253.0]","[1167.0, 353.0]","[1142.0, 478.0]",914.0,79.0,1031.0,237.0,962.2409,80.03060,804.0,711.0,816.0,510.0,908.0,438.0,1040.0,454.0,906.0,528.0,883.0,707.0,869.0,214.0,798.0,340.0,902.0,253.0,1067.0,253.0,1167.0,353.0,1142.0,478.0,12
9,7,0,021233911.jpg,1,"[374.0, 87.0, 467.0, 193.0]","{'6': 1, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [405.0, 302.0], 7: [404.0, 201.0], 8: [405...",1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,"[405.0, 302.0]","[404.0, 201.0]","[405.772, 194.4489]","[435.228, 85.5511]","[357.0, 353.0]","[369.0, 330.0]","[366.0, 296.0]","[443.0, 308.0]","[470.0, 332.0]","[435.0, 371.0]","[331.0, 252.0]","[354.0, 261.0]","[357.0, 200.0]","[451.0, 202.0]","[484.0, 246.0]","[468.0, 253.0]",374.0,87.0,467.0,193.0,435.2280,85.55110,357.0,353.0,369.0,330.0,366.0,296.0,443.0,308.0,470.0,332.0,435.0,371.0,331.0,252.0,354.0,261.0,357.0,200.0,451.0,202.0,484.0,246.0,468.0,253.0,12
10,8,0,018182497.jpg,1,"[256.0, 99.0, 346.0, 215.0]","{'6': 1, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [356.0, 290.0], 7: [314.0, 209.0], 8: [315...",1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,"[356.0, 290.0]","[314.0, 209.0]","[315.2436, 213.9744]","[286.7564, 100.0256]","[325.0, 349.0]","[341.0, 327.0]","[326.0, 302.0]","[385.0, 277.0]","[409.0, 302.0]","[399.0, 377.0]","[318.0, 325.0]","[280.0, 288.0]","[267.0, 231.0]","[360.0, 187.0]","[406.0, 218.0]","[374.0, 258.0]",256.0,99.0,346.0,215.0,286.7564,100.02560,325.0,349.0,341.0,327.0,326.0,302.0,385.0,277.0,409.0,302.0,399.0,377.0,318.0,325.0,280.0,288.0,267.0,231.0,360.0,187.0,406.0,218.0,374.0,258.0,12
12,9,1,018340451.jpg,1,"[332.0, 125.0, 376.0, 186.0]","{'6': 1, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [369.0, 255.0], 7: [344.0, 190.0], 8: [345...",1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,"[369.0, 255.0]","[344.0, 190.0]","[345.6244, 184.3958]","[362.3756, 126.6042]","[423.0, 378.0]","[411.0, 288.0]","[344.0, 259.0]","[394.0, 251.0]","[456.0, 283.0]","[453.0, 369.0]","[381.0, 264.0]","[337.0, 247.0]","[316.0, 189.0]","[372.0, 190.0]","[394.0, 239.0]","[421.0, 257.0]",332.0,125.0,376.0,186.0,362.3756,126.60420,423.0,378.0,411.0,288.0,344.0,259.0,394.0,251.0,456.0,283.0,453.0,369.0,381.0,264.0,337.0,247.0,316.0,189.0,372.0,190.0,394.0,239.0,421.0,257.0,12
13,10,0,030424224.jpg,1,"[327.0, 95.0, 372.0, 160.0]","{'6': 1, '7': 1, '8': 0, '9': 0, '0': 1, '1': ...","{6: [324.0, 265.0], 7: [334.0, 170.0], 8: [338...",1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,"[324.0, 265.0]","[334.0, 170.0]","[338.6651, 157.2087]","[360.3349, 97.7913]","[324.0, 406.0]"

In [ ]:
# dropping unnecessary columns

df = df.drop(['level_0', 'level_1', 'train', 'head_rect', 'is_visible', 'joint_pos', 'pelvis', 'thorax', 'upper_neck', 'joint_pos_pelvis', 'joint_pos_thorax', 'joint_pos_upper_neck', 'joint_pos_head_top', 'joint_pos_r_ankle', 'joint_pos_r_knee', 'joint_pos_r_hip', 'joint_pos_l_hip', 'joint_pos_l_knee', 'joint_pos_l_ankle', 'joint_pos_r_wrist', 'joint_pos_r_elbow', 'joint_pos_r_shoulder', 'joint_pos_l_shoulder', 'joint_pos_l_elbow', 'joint_pos_l_wrist', 'total'], axis=1)
df

,filename,head_top,r_ankle,r_knee,r_hip,l_hip,l_knee,l_ankle,r_wrist,r_elbow,r_shoulder,l_shoulder,l_elbow,l_wrist,head_rect_x1,head_rect_y1,head_rect_x2,head_rect_y2,joint_pos_head_top_x,joint_pos_head_top_y,joint_pos_r_ankle_x,joint_pos_r_ankle_y,joint_pos_r_knee_x,joint_pos_r_knee_y,joint_pos_r_hip_x,joint_pos_r_hip_y,joint_pos_l_hip_x,joint_pos_l_hip_y,joint_pos_l_knee_x,joint_pos_l_knee_y,joint_pos_l_ankle_x,joint_pos_l_ankle_y,joint_pos_r_wrist_x,joint_pos_r_wrist_y,joint_pos_r_elbow_x,joint_pos_r_elbow_y,joint_pos_r_shoulder_x,joint_pos_r_shoulder_y,joint_pos_l_shoulder_x,joint_pos_l_shoulder_y,joint_pos_l_elbow_x,joint_pos_l_elbow_y,joint_pos_l_wrist_x,joint_pos_l_wrist_y
2,005808361.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,914.0,79.0,1031.0,237.0,962.2409,80.03060,804.0,711.0,816.0,510.0,908.0,438.0,1040.0,454.0,906.0,528.0,883.0,707.0,869.0,214.0,798.0,340.0,902.0,253.0,1067.0,253.0,1167.0,353.0,1142.0,478.0
9,021233911.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,374.0,87.0,467.0,193.0,435.2280,85.55110,357.0,353.0,369.0,330.0,366.0,296.0,443.0,308.0,470.0,332.0,435.0,371.0,331.0,252.0,354.0,261.0,357.0,200.0,451.0,202.0,484.0,246.0,468.0,253.0
10,018182497.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,256.0,99.0,346.0,215.0,286.7564,100.02560,325.0,349.0,341.0,327.0,326.0,302.0,385.0,277.0,409.0,302.0,399.0,377.0,318.0,325.0,280.0,288.0,267.0,231.0,360.0,187.0,406.0,218.0,374.0,258.0
12,018340451.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,332.0,125.0,376.0,186.0,362.3756,126.60420,423.0,378.0,411.0,288.0,344.0,259.0,394.0,251.0,456.0,283.0,453.0,369.0,381.0,264.0,337.0,247.0,316.0,189.0,372.0,190.0,394.0,239.0,421.0,257.0
13,030424224.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,327.0,95.0,372.0,160.0,360.3349,97.79130,324.0,406.0,322.0,291.0,293.0,262.0,355.0,267.0,401.0,292.0,394.0,404.0,363.0,167.0,319.0,205.0,298.0,168.0,369.0,172.0,390.0,221.0,398.0,179.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19827,098128966.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,367.0,100.0,398.0,153.0,382.0367,101.94420,362.0,414.0,357.0,342.0,361.0,259.0,395.0,257.0,404.0,324.0,393.0,404.0,336.0,208.0,329.0,173.0,350.0,154.0,416.0,152.0,444.0,169.0,428.0,224.0
19846,056896314.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,1094.0,199.0,1126.0,245.0,1120.8853,202.40640,990.0,386.0,1015.0,346.0,1041.0,309.0,1062.0,327.0,1042.0,362.0,1019.0,391.0,1027.0,292.0,1047.0,257.0,1073.0,226.0,1116.0,271.0,1103.0,309.0,1082.0,345.0
19848,056896314.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,1227.0,297.0,1255.0,333.0,1248.1081,298.19900,1139.0,458.0,1161.0,422.0,1183.0,385.0,1200.0,404.0,1179.0,440.0,1159.0,472.0,1183.0,364.0,1198.0,337.0,1214.0,321.0,1245.0,361.0,1233.0,390.0,1214.0,420.0
19877,024009590.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,973.0,341.0,999.0,383.0,987.8443,342.32770,922.0,553.0,932.0,490.0,966.0,457.0,992.0,457.0,1027.0,504.0,1020.0,572.0,958.0,448.0,933.0,428.0,954.0,392.0,1011.0,396.0,1028.0,431.0,1017.0,467.0


In [ ]:
df.round()
df

,filename,head_top,r_ankle,r_knee,r_hip,l_hip,l_knee,l_ankle,r_wrist,r_elbow,r_shoulder,l_shoulder,l_elbow,l_wrist,head_rect_x1,head_rect_y1,head_rect_x2,head_rect_y2,joint_pos_head_top_x,joint_pos_head_top_y,joint_pos_r_ankle_x,joint_pos_r_ankle_y,joint_pos_r_knee_x,joint_pos_r_knee_y,joint_pos_r_hip_x,joint_pos_r_hip_y,joint_pos_l_hip_x,joint_pos_l_hip_y,joint_pos_l_knee_x,joint_pos_l_knee_y,joint_pos_l_ankle_x,joint_pos_l_ankle_y,joint_pos_r_wrist_x,joint_pos_r_wrist_y,joint_pos_r_elbow_x,joint_pos_r_elbow_y,joint_pos_r_shoulder_x,joint_pos_r_shoulder_y,joint_pos_l_shoulder_x,joint_pos_l_shoulder_y,joint_pos_l_elbow_x,joint_pos_l_elbow_y,joint_pos_l_wrist_x,joint_pos_l_wrist_y
2,005808361.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,914.0,79.0,1031.0,237.0,962.2409,80.03060,804.0,711.0,816.0,510.0,908.0,438.0,1040.0,454.0,906.0,528.0,883.0,707.0,869.0,214.0,798.0,340.0,902.0,253.0,1067.0,253.0,1167.0,353.0,1142.0,478.0
9,021233911.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,374.0,87.0,467.0,193.0,435.2280,85.55110,357.0,353.0,369.0,330.0,366.0,296.0,443.0,308.0,470.0,332.0,435.0,371.0,331.0,252.0,354.0,261.0,357.0,200.0,451.0,202.0,484.0,246.0,468.0,253.0
10,018182497.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,256.0,99.0,346.0,215.0,286.7564,100.02560,325.0,349.0,341.0,327.0,326.0,302.0,385.0,277.0,409.0,302.0,399.0,377.0,318.0,325.0,280.0,288.0,267.0,231.0,360.0,187.0,406.0,218.0,374.0,258.0
12,018340451.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,332.0,125.0,376.0,186.0,362.3756,126.60420,423.0,378.0,411.0,288.0,344.0,259.0,394.0,251.0,456.0,283.0,453.0,369.0,381.0,264.0,337.0,247.0,316.0,189.0,372.0,190.0,394.0,239.0,421.0,257.0
13,030424224.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,327.0,95.0,372.0,160.0,360.3349,97.79130,324.0,406.0,322.0,291.0,293.0,262.0,355.0,267.0,401.0,292.0,394.0,404.0,363.0,167.0,319.0,205.0,298.0,168.0,369.0,172.0,390.0,221.0,398.0,179.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19827,098128966.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,367.0,100.0,398.0,153.0,382.0367,101.94420,362.0,414.0,357.0,342.0,361.0,259.0,395.0,257.0,404.0,324.0,393.0,404.0,336.0,208.0,329.0,173.0,350.0,154.0,416.0,152.0,444.0,169.0,428.0,224.0
19846,056896314.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,1094.0,199.0,1126.0,245.0,1120.8853,202.40640,990.0,386.0,1015.0,346.0,1041.0,309.0,1062.0,327.0,1042.0,362.0,1019.0,391.0,1027.0,292.0,1047.0,257.0,1073.0,226.0,1116.0,271.0,1103.0,309.0,1082.0,345.0
19848,056896314.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,1227.0,297.0,1255.0,333.0,1248.1081,298.19900,1139.0,458.0,1161.0,422.0,1183.0,385.0,1200.0,404.0,1179.0,440.0,1159.0,472.0,1183.0,364.0,1198.0,337.0,1214.0,321.0,1245.0,361.0,1233.0,390.0,1214.0,420.0
19877,024009590.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,973.0,341.0,999.0,383.0,987.8443,342.32770,922.0,553.0,932.0,490.0,966.0,457.0,992.0,457.0,1027.0,504.0,1020.0,572.0,958.0,448.0,933.0,428.0,954.0,392.0,1011.0,396.0,1028.0,431.0,1017.0,467.0


In [1]:
# dropping columns representing presence of a keypoint
# t_df = df
# t_df = t_df.drop(['head_top', 'head_rect_x1', 'head_rect_y1', 'head_rect_x2', 'head_rect_y2', 'joint_pos_head_top_x', 'joint_pos_head_top_y'], axis=1)
# t_df.columns
# t_df

In [ ]:
# generate csv
df.to_csv("anno_list.csv", index=False)

In [ ]:
# df = pd.read_csv('anno_list.csv')

In [ ]:
df

,filename,head_top,r_ankle,r_knee,r_hip,l_hip,l_knee,l_ankle,r_wrist,r_elbow,r_shoulder,l_shoulder,l_elbow,l_wrist,head_rect_x1,head_rect_y1,head_rect_x2,head_rect_y2,joint_pos_head_top_x,joint_pos_head_top_y,joint_pos_r_ankle_x,joint_pos_r_ankle_y,joint_pos_r_knee_x,joint_pos_r_knee_y,joint_pos_r_hip_x,joint_pos_r_hip_y,joint_pos_l_hip_x,joint_pos_l_hip_y,joint_pos_l_knee_x,joint_pos_l_knee_y,joint_pos_l_ankle_x,joint_pos_l_ankle_y,joint_pos_r_wrist_x,joint_pos_r_wrist_y,joint_pos_r_elbow_x,joint_pos_r_elbow_y,joint_pos_r_shoulder_x,joint_pos_r_shoulder_y,joint_pos_l_shoulder_x,joint_pos_l_shoulder_y,joint_pos_l_elbow_x,joint_pos_l_elbow_y,joint_pos_l_wrist_x,joint_pos_l_wrist_y
0,005808361.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,914.0,79.0,1031.0,237.0,962.2409,80.03060,804.0,711.0,816.0,510.0,908.0,438.0,1040.0,454.0,906.0,528.0,883.0,707.0,869.0,214.0,798.0,340.0,902.0,253.0,1067.0,253.0,1167.0,353.0,1142.0,478.0
1,021233911.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,374.0,87.0,467.0,193.0,435.2280,85.55110,357.0,353.0,369.0,330.0,366.0,296.0,443.0,308.0,470.0,332.0,435.0,371.0,331.0,252.0,354.0,261.0,357.0,200.0,451.0,202.0,484.0,246.0,468.0,253.0
2,018182497.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,256.0,99.0,346.0,215.0,286.7564,100.02560,325.0,349.0,341.0,327.0,326.0,302.0,385.0,277.0,409.0,302.0,399.0,377.0,318.0,325.0,280.0,288.0,267.0,231.0,360.0,187.0,406.0,218.0,374.0,258.0
3,018340451.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,332.0,125.0,376.0,186.0,362.3756,126.60420,423.0,378.0,411.0,288.0,344.0,259.0,394.0,251.0,456.0,283.0,453.0,369.0,381.0,264.0,337.0,247.0,316.0,189.0,372.0,190.0,394.0,239.0,421.0,257.0
4,030424224.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,327.0,95.0,372.0,160.0,360.3349,97.79130,324.0,406.0,322.0,291.0,293.0,262.0,355.0,267.0,401.0,292.0,394.0,404.0,363.0,167.0,319.0,205.0,298.0,168.0,369.0,172.0,390.0,221.0,398.0,179.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3831,098128966.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,367.0,100.0,398.0,153.0,382.0367,101.94420,362.0,414.0,357.0,342.0,361.0,259.0,395.0,257.0,404.0,324.0,393.0,404.0,336.0,208.0,329.0,173.0,350.0,154.0,416.0,152.0,444.0,169.0,428.0,224.0
3832,056896314.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,1094.0,199.0,1126.0,245.0,1120.8853,202.40640,990.0,386.0,1015.0,346.0,1041.0,309.0,1062.0,327.0,1042.0,362.0,1019.0,391.0,1027.0,292.0,1047.0,257.0,1073.0,226.0,1116.0,271.0,1103.0,309.0,1082.0,345.0
3833,056896314.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,1227.0,297.0,1255.0,333.0,1248.1081,298.19900,1139.0,458.0,1161.0,422.0,1183.0,385.0,1200.0,404.0,1179.0,440.0,1159.0,472.0,1183.0,364.0,1198.0,337.0,1214.0,321.0,1245.0,361.0,1233.0,390.0,1214.0,420.0
3834,024009590.jpg,0,1,1,1,1,1,1,1,1,1,1,1,1,973.0,341.0,999.0,383.0,987.8443,342.32770,922.0,553.0,932.0,490.0,966.0,457.0,992.0,457.0,1027.0,504.0,1020.0,572.0,958.0,448.0,933.0,428.0,954.0,392.0,1011.0,396.0,1028.0,431.0,1017.0,467.0


In [ ]:
# drop columns representing presence of keypoints
# df = df.drop(columns=['r_ankle', 'r_knee', 'r_hip', 'l_hip', 'l_knee', 'l_ankle', 'r_wrist', 'r_elbow', 'r_shoulder', 'l_shoulder', 'l_elbow', 'l_wrist', 'total'], axis=1)
df = df.drop(columns=['head_top','r_ankle', 'r_knee', 'r_hip', 'l_hip', 'l_knee', 'l_ankle', 'r_wrist', 'r_elbow', 'r_shoulder', 'l_shoulder', 'l_elbow', 'l_wrist'], axis=1)
df

In [ ]:
temp_df = pd.read_csv('data/stage2_updated.csv')

widths = temp_df.width
heights = temp_df.height

del temp_df

In [ ]:
# check width and height of each image
for w, h in zip(widths, heights):
  print(w, h)

1280 720
640 480
640 480
854 480
854 480
854 480
854 480
854 480
720 480
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
854 480
854 480
1920 1080
640 480
640 480
1280 720
1280 720
1280 720
640 480
854 468
854 468
854 468
854 468
854 468
854 468
854 468
854 468
854 468
1440 1080
1280 720
1280 720
1920 1080
1920 1080
1920 1080
1920 1080
1920 1080
854 480
854 480
854 480
1920 1080
1920 1080
1920 1080
1920 1080
1920 1080
1920 1080
1920 1080
1920 1080
1920 1080
1920 1080
1920 1080
1920 1080
1920 1080
1280 720
1920 1080
1280 720
1280 720
1280 720
640 480
640 480
854 480
854 480
640 480
1280 720
1920 1080
1920 1080
1920 1080
1920 1080
1920 1080
1920 1080
1920 1080
1920 1080
640 480
720 480
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
640 480
640 480
640 480
640 480
640 480
640 480
1280 720
1280 720
1920 1080
1920 1080
640 480
1280 720
1920 1080
1920 1080
1280 720
1920 1080
1920 1080
1920 1080
1280 720
1280

In [ ]:
df["width"] = widths
df["height"] = heights

In [ ]:
df["head_length"] = ((df.head_rect_x2 - df.head_rect_x1) + (df.head_rect_y2 - df.head_rect_y1)) // 2 

In [ ]:
x_cols = ["joint_pos_r_ankle_x", "joint_pos_r_knee_x", "joint_pos_r_hip_x", "joint_pos_l_hip_x", "joint_pos_l_knee_x", "joint_pos_l_ankle_x", "joint_pos_r_wrist_x", "joint_pos_r_elbow_x", "joint_pos_r_shoulder_x", "joint_pos_l_shoulder_x", "joint_pos_l_elbow_x", "joint_pos_l_wrist_x"]
y_cols = ["joint_pos_r_ankle_y", "joint_pos_r_knee_y", "joint_pos_r_hip_y", "joint_pos_l_hip_y", "joint_pos_l_knee_y", "joint_pos_l_ankle_y", "joint_pos_r_wrist_y", "joint_pos_r_elbow_y", "joint_pos_r_shoulder_y", "joint_pos_l_shoulder_y", "joint_pos_l_elbow_y", "joint_pos_l_wrist_y"]

In [ ]:
# calculating bounding box for a person in each image using coordinates of keypoints and buffer

temp_x_min = np.array(df[x_cols].min(axis = 1) - BUFFER - df.head_length)
temp_y_min = np.array(df[y_cols].min(axis = 1) - BUFFER - df.head_length)

temp_x_max = np.array(df[x_cols].max(axis = 1) + BUFFER + df.head_length)
temp_y_max = np.array(df[y_cols].max(axis = 1) + BUFFER + df.head_length)

cnt =0

for i in range(temp_x_min.shape[0]):
  if temp_x_min[i] < 0:
    temp_x_min[i] = 0
    cnt+=1
  if temp_y_min[i] < 0:
    temp_y_min[i] = 0
    cnt+=1
print(cnt, "negative values filtered")


cnt =0
for i in range(temp_x_max.shape[0]):
  if temp_x_max[i] >= widths[i]:
    temp_x_max[i] = widths[i]-1
    cnt+=1
  if temp_y_max[i] >= heights[i]:
    temp_y_max[i] = heights[i] -1
    cnt+=1
print(cnt, "out-of-bound-values-filtered values filtered")

df["x_max"] = temp_x_max
df["y_max"] = temp_y_max


df["x_min"] = temp_x_min
df["y_min"] = temp_y_min

df

222 negative values filtered
877 out-of-bound-values-filtered values filtered


,filename,head_rect_x1,head_rect_y1,head_rect_x2,head_rect_y2,joint_pos_head_top_x,joint_pos_head_top_y,joint_pos_r_ankle_x,joint_pos_r_ankle_y,joint_pos_r_knee_x,joint_pos_r_knee_y,joint_pos_r_hip_x,joint_pos_r_hip_y,joint_pos_l_hip_x,joint_pos_l_hip_y,joint_pos_l_knee_x,joint_pos_l_knee_y,joint_pos_l_ankle_x,joint_pos_l_ankle_y,joint_pos_r_wrist_x,joint_pos_r_wrist_y,joint_pos_r_elbow_x,joint_pos_r_elbow_y,joint_pos_r_shoulder_x,joint_pos_r_shoulder_y,joint_pos_l_shoulder_x,joint_pos_l_shoulder_y,joint_pos_l_elbow_x,joint_pos_l_elbow_y,joint_pos_l_wrist_x,joint_pos_l_wrist_y,width,height,head_length,x_max,y_max,x_min,y_min
0,005808361.jpg,914.0,79.0,1031.0,237.0,962.2409,80.03060,804.0,711.0,816.0,510.0,908.0,438.0,1040.0,454.0,906.0,528.0,883.0,707.0,869.0,214.0,798.0,340.0,902.0,253.0,1067.0,253.0,1167.0,353.0,1142.0,478.0,1280,720,137.0,1279.0,719.0,651.0,67.0
1,021233911.jpg,374.0,87.0,467.0,193.0,435.2280,85.55110,357.0,353.0,369.0,330.0,366.0,296.0,443.0,308.0,470.0,332.0,435.0,371.0,331.0,252.0,354.0,261.0,357.0,200.0,451.0,202.0,484.0,246.0,468.0,253.0,640,480,99.0,593.0,479.0,222.0,91.0
2,018182497.jpg,256.0,99.0,346.0,215.0,286.7564,100.02560,325.0,349.0,341.0,327.0,326.0,302.0,385.0,277.0,409.0,302.0,399.0,377.0,318.0,325.0,280.0,288.0,267.0,231.0,360.0,187.0,406.0,218.0,374.0,258.0,640,480,103.0,522.0,479.0,154.0,74.0
3,018340451.jpg,332.0,125.0,376.0,186.0,362.3756,126.60420,423.0,378.0,411.0,288.0,344.0,259.0,394.0,251.0,456.0,283.0,453.0,369.0,381.0,264.0,337.0,247.0,316.0,189.0,372.0,190.0,394.0,239.0,421.0,257.0,854,480,52.0,518.0,440.0,254.0,127.0
4,030424224.jpg,327.0,95.0,372.0,160.0,360.3349,97.79130,324.0,406.0,322.0,291.0,293.0,262.0,355.0,267.0,401.0,292.0,394.0,404.0,363.0,167.0,319.0,205.0,298.0,168.0,369.0,172.0,390.0,221.0,398.0,179.0,854,480,55.0,466.0,471.0,228.0,102.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3831,098128966.jpg,367.0,100.0,398.0,153.0,382.0367,101.94420,362.0,414.0,357.0,342.0,361.0,259.0,395.0,257.0,404.0,324.0,393.0,404.0,336.0,208.0,329.0,173.0,350.0,154.0,416.0,152.0,444.0,169.0,428.0,224.0,654,480,42.0,496.0,466.0,277.0,100.0
3832,056896314.jpg,1094.0,199.0,1126.0,245.0,1120.8853,202.40640,990.0,386.0,1015.0,346.0,1041.0,309.0,1062.0,327.0,1042.0,362.0,1019.0,391.0,1027.0,292.0,1047.0,257.0,1073.0,226.0,1116.0,271.0,1103.0,309.0,1082.0,345.0,1280,720,39.0,1165.0,440.0,941.0,177.0
3833,056896314.jpg,1227.0,297.0,1255.0,333.0,1248.1081,298.19900,1139.0,458.0,1161.0,422.0,1183.0,385.0,1200.0,404.0,1179.0,440.0,1159.0,472.0,1183.0,364.0,1198.0,337.0,1214.0,321.0,1245.0,361.0,1233.0,390.0,1214.0,420.0,1280,720,32.0,1279.0,514.0,1097.0,279.0
3834,024009590.jpg,973.0,341.0,999.0,383.0,987.8443,342.32770,922.0,553.0,932.0,490.0,966.0,457.0,992.0,457.0,1027.0,504.0,1020.0,572.0,958.0,448.0,933.0,428.0,954.0,392.0,1011.0,396.0,1028.0,431.0,1017.0,467.0,1920,1080,34.0,1072.0,616.0,878.0,348.0


In [ ]:
final_df = pd.DataFrame(data={"filename" :df["filename"], "x_min" : df["x_min"], "y_min" : df["y_min"], "x_max" : df["x_max"], "y_max" : df["y_max"], "head_length" : df["head_length"]} )

final_df["width"] = widths
final_df["height"] = heights

for x in x_cols:
  final_df[x] = df[x] - df['x_min']

for y in y_cols:
  final_df[y] = df[y] - df["y_min"]

final_df.head()

,filename,x_min,y_min,x_max,y_max,head_length,width,height,joint_pos_r_ankle_x,joint_pos_r_knee_x,joint_pos_r_hip_x,joint_pos_l_hip_x,joint_pos_l_knee_x,joint_pos_l_ankle_x,joint_pos_r_wrist_x,joint_pos_r_elbow_x,joint_pos_r_shoulder_x,joint_pos_l_shoulder_x,joint_pos_l_elbow_x,joint_pos_l_wrist_x,joint_pos_r_ankle_y,joint_pos_r_knee_y,joint_pos_r_hip_y,joint_pos_l_hip_y,joint_pos_l_knee_y,joint_pos_l_ankle_y,joint_pos_r_wrist_y,joint_pos_r_elbow_y,joint_pos_r_shoulder_y,joint_pos_l_shoulder_y,joint_pos_l_elbow_y,joint_pos_l_wrist_y
0,005808361.jpg,651.0,67.0,1279.0,719.0,137.0,1280,720,153.0,165.0,257.0,389.0,255.0,232.0,218.0,147.0,251.0,416.0,516.0,491.0,644.0,443.0,371.0,387.0,461.0,640.0,147.0,273.0,186.0,186.0,286.0,411.0
1,021233911.jpg,222.0,91.0,593.0,479.0,99.0,640,480,135.0,147.0,144.0,221.0,248.0,213.0,109.0,132.0,135.0,229.0,262.0,246.0,262.0,239.0,205.0,217.0,241.0,280.0,161.0,170.0,109.0,111.0,155.0,162.0
2,018182497.jpg,154.0,74.0,522.0,479.0,103.0,640,480,171.0,187.0,172.0,231.0,255.0,245.0,164.0,126.0,113.0,206.0,252.0,220.0,275.0,253.0,228.0,203.0,228.0,303.0,251.0,214.0,157.0,113.0,144.0,184.0
3,018340451.jpg,254.0,127.0,518.0,440.0,52.0,854,480,169.0,157.0,90.0,140.0,202.0,199.0,127.0,83.0,62.0,118.0,140.0,167.0,251.0,161.0,132.0,124.0,156.0,242.0,137.0,120.0,62.0,63.0,112.0,130.0
4,030424224.jpg,228.0,102.0,466.0,471.0,55.0,854,480,96.0,94.0,65.0,127.0,173.0,166.0,135.0,91.0,70.0,141.0,162.0,170.0,304.0,189.0,160.0,165.0,190.0,302.0,65.0,103.0,66.0,70.0,119.0,77.0


In [ ]:
# final_df["head_length"] = ((df.head_rect_x2 - df.head_rect_x1) + (df.head_rect_y2 - df.head_rect_y1)) // 2 

In [ ]:
# generate csv for the final dataframe

final_df.to_csv(os.path.join("data", "stage2_updated.csv"), index = False)

In [ ]:
final_df

,filename,x_min,y_min,x_max,y_max,head_length,width,height,joint_pos_r_ankle_x,joint_pos_r_knee_x,joint_pos_r_hip_x,joint_pos_l_hip_x,joint_pos_l_knee_x,joint_pos_l_ankle_x,joint_pos_r_wrist_x,joint_pos_r_elbow_x,joint_pos_r_shoulder_x,joint_pos_l_shoulder_x,joint_pos_l_elbow_x,joint_pos_l_wrist_x,joint_pos_r_ankle_y,joint_pos_r_knee_y,joint_pos_r_hip_y,joint_pos_l_hip_y,joint_pos_l_knee_y,joint_pos_l_ankle_y,joint_pos_r_wrist_y,joint_pos_r_elbow_y,joint_pos_r_shoulder_y,joint_pos_l_shoulder_y,joint_pos_l_elbow_y,joint_pos_l_wrist_y
0,005808361.jpg,651.0,67.0,1279.0,719.0,137.0,1280,720,153.0,165.0,257.0,389.0,255.0,232.0,218.0,147.0,251.0,416.0,516.0,491.0,644.0,443.0,371.0,387.0,461.0,640.0,147.0,273.0,186.0,186.0,286.0,411.0
1,021233911.jpg,222.0,91.0,593.0,479.0,99.0,640,480,135.0,147.0,144.0,221.0,248.0,213.0,109.0,132.0,135.0,229.0,262.0,246.0,262.0,239.0,205.0,217.0,241.0,280.0,161.0,170.0,109.0,111.0,155.0,162.0
2,018182497.jpg,154.0,74.0,522.0,479.0,103.0,640,480,171.0,187.0,172.0,231.0,255.0,245.0,164.0,126.0,113.0,206.0,252.0,220.0,275.0,253.0,228.0,203.0,228.0,303.0,251.0,214.0,157.0,113.0,144.0,184.0
3,018340451.jpg,254.0,127.0,518.0,440.0,52.0,854,480,169.0,157.0,90.0,140.0,202.0,199.0,127.0,83.0,62.0,118.0,140.0,167.0,251.0,161.0,132.0,124.0,156.0,242.0,137.0,120.0,62.0,63.0,112.0,130.0
4,030424224.jpg,228.0,102.0,466.0,471.0,55.0,854,480,96.0,94.0,65.0,127.0,173.0,166.0,135.0,91.0,70.0,141.0,162.0,170.0,304.0,189.0,160.0,165.0,190.0,302.0,65.0,103.0,66.0,70.0,119.0,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3831,098128966.jpg,277.0,100.0,496.0,466.0,42.0,654,480,85.0,80.0,84.0,118.0,127.0,116.0,59.0,52.0,73.0,139.0,167.0,151.0,314.0,242.0,159.0,157.0,224.0,304.0,108.0,73.0,54.0,52.0,69.0,124.0
3832,056896314.jpg,941.0,177.0,1165.0,440.0,39.0,1280,720,49.0,74.0,100.0,121.0,101.0,78.0,86.0,106.0,132.0,175.0,162.0,141.0,209.0,169.0,132.0,150.0,185.0,214.0,115.0,80.0,49.0,94.0,132.0,168.0
3833,056896314.jpg,1097.0,279.0,1279.0,514.0,32.0,1280,720,42.0,64.0,86.0,103.0,82.0,62.0,86.0,101.0,117.0,148.0,136.0,117.0,179.0,143.0,106.0,125.0,161.0,193.0,85.0,58.0,42.0,82.0,111.0,141.0
3834,024009590.jpg,878.0,348.0,1072.0,616.0,34.0,1920,1080,44.0,54.0,88.0,114.0,149.0,142.0,80.0,55.0,76.0,133.0,150.0,139.0,205.0,142.0,109.0,109.0,156.0,224.0,100.0,80.0,44.0,48.0,83.0,119.0


In [ ]:
# Deleting unrequisite images from images folder 

directory = "./data/mpii_human_pose/images"
filenames = final_df['filename'].to_list()

# count = 0
for filename in os.listdir(directory):
  file_path = os.path.join(directory, filename)
  try:
    if filename not in filenames:
      os.remove(file_path)
      print("Removed image: %s", filename)
  except OSError:
    print("Already removed image %s", filename)
    # print(os.path.join(directory, filename))

# count

Streaming output truncated to the last 5000 lines.
Removed image: %s 081298574.jpg
Removed image: %s 089827266.jpg
Removed image: %s 058903894.jpg
Removed image: %s 011986537.jpg
Removed image: %s 012822995.jpg
Removed image: %s 071812308.jpg
Removed image: %s 084913225.jpg
Removed image: %s 066995349.jpg
Removed image: %s 037208225.jpg
Removed image: %s 078969451.jpg
Removed image: %s 073257210.jpg
Removed image: %s 069551048.jpg
Removed image: %s 095552370.jpg
Removed image: %s 026518088.jpg
Removed image: %s 053263274.jpg
Removed image: %s 097098406.jpg
Removed image: %s 021494038.jpg
Removed image: %s 000040154.jpg
Removed image: %s 083524555.jpg
Removed image: %s 087470198.jpg
Removed image: %s 005637550.jpg
Removed image: %s 075982366.jpg
Removed image: %s 065287309.jpg
Removed image: %s 032709151.jpg
Removed image: %s 070328005.jpg
Removed image: %s 080589117.jpg
Removed image: %s 001823258.jpg
Removed image: %s 019387640.jpg
Removed image: %s 048806016.jpg
Removed image: %s 017

In [ ]:
# count the number of images in directory and in the dataframe to confirm
count = 0
for filename in os.listdir(directory):
  count = count+1

print(count)

print(len(final_df['filename'].unique().tolist()))

3064